In [2]:
import src.control.openinterface.commands as commands

In [5]:
commands.schedule()

b'\xa7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [7]:
bytes([128])+bytes([128])

b'\x80\x80'

In [8]:
sum(bytes([128,128]))

256

In [13]:
class Parent:
    y = 3
    def __init__(self):
        x = 7
        
    def pfunc(self):
        return self.y

class Child(Parent):
    y = 5
    def __init__(self):
        super().__init__()

In [14]:
x = Child()

In [15]:
x.pfunc()

5

In [2]:
import src.control.openinterface.packets as packets

In [3]:
fct = packets.SensorPacketFactory()

In [5]:
len(fct.classes)

49

In [6]:
[0,1,2,3,4,5,6,7,8,9][2:4]

[2, 3]

In [17]:
x = bytes([128,128])

In [18]:
bool(x)

True